In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_FV
from utils_training import FVTrainingData
from utils_training import evaluate_FVmodel

In [4]:
from behavior_model import FVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 4, 0.25
acc_std_dev = 0.07
Inputs, Inputs_Vl, Outputs, Weights = FVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_05_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Inputs_Vl_train = [Inputs_Vl[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Inputs_Vl_val = [Inputs_Vl[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_FV(Inputs_train, Inputs_Vl_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_FV(Inputs_val, Inputs_Vl_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = FVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_fv_val_05.pth'
best_train_model_path = f'.\\weights\\bc_fv_train_05.pth'
best_trainval_model_path = f'.\\weights\\bc_fv_trainval_05.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, inputs_vl, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        inputs_vl = inputs_vl.reshape(inputs_vl.shape[0],1,inputs_vl.shape[1])
        inputs_vl = inputs_vl.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs,inputs_vl)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_FVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_FVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 353.63,      Loss_min: 346.99, Loss_max: 359.84 
      Loss_mean_val: 352.69, Loss_min_val: 321.97,      Loss_max_val: 367.47 
      Loss_mean_trainval: 352.79
Epoch 2/4000, Loss_mean: 327.71,      Loss_min: 320.48, Loss_max: 336.38 
      Loss_mean_val: 323.99, Loss_min_val: 286.0,      Loss_max_val: 348.01 
      Loss_mean_trainval: 324.39
Epoch 3/4000, Loss_mean: 304.69,      Loss_min: 282.28, Loss_max: 316.2 
      Loss_mean_val: 303.73, Loss_min_val: 219.85,      Loss_max_val: 344.96 
      Loss_mean_trainval: 303.83
Epoch 4/4000, Loss_mean: 289.85,      Loss_min: 236.81, Loss_max: 332.66 
      Loss_mean_val: 288.35, Loss_min_val: 190.81,      Loss_max_val: 347.19 
      Loss_mean_trainval: 288.51
Epoch 5/4000, Loss_mean: 285.25,      Loss_min: 271.07, Loss_max: 299.38 
      Loss_mean_val: 282.89, Loss_min_val: 215.61,      Loss_max_val: 328.46 
      Loss_mean_trainval: 283.14
Epoch 6/4000, Loss_mean: 276.83,      Loss_min: 247.59, Loss_max: 304.99 
   

Epoch 46/4000, Loss_mean: 138.9,      Loss_min: 89.53, Loss_max: 180.38 
      Loss_mean_val: 146.57, Loss_min_val: 98.34,      Loss_max_val: 228.55 
      Loss_mean_trainval: 145.75
Epoch 47/4000, Loss_mean: 138.11,      Loss_min: 111.53, Loss_max: 170.9 
      Loss_mean_val: 143.03, Loss_min_val: 86.37,      Loss_max_val: 189.35 
      Loss_mean_trainval: 142.51
Epoch 48/4000, Loss_mean: 131.66,      Loss_min: 106.81, Loss_max: 150.63 
      Loss_mean_val: 146.39, Loss_min_val: 102.82,      Loss_max_val: 202.02 
      Loss_mean_trainval: 144.82
Epoch 49/4000, Loss_mean: 134.85,      Loss_min: 102.87, Loss_max: 178.03 
      Loss_mean_val: 142.38, Loss_min_val: 95.49,      Loss_max_val: 212.73 
      Loss_mean_trainval: 141.57
Epoch 50/4000, Loss_mean: 127.05,      Loss_min: 101.4, Loss_max: 149.15 
      Loss_mean_val: 144.48, Loss_min_val: 108.56,      Loss_max_val: 195.11 
      Loss_mean_trainval: 142.61
Epoch 51/4000, Loss_mean: 130.16,      Loss_min: 100.88, Loss_max: 152.62 
  

Epoch 91/4000, Loss_mean: 96.63,      Loss_min: 64.18, Loss_max: 120.23 
      Loss_mean_val: 120.93, Loss_min_val: 70.99,      Loss_max_val: 174.37 
      Loss_mean_trainval: 118.33
Epoch 92/4000, Loss_mean: 101.4,      Loss_min: 80.16, Loss_max: 126.79 
      Loss_mean_val: 120.05, Loss_min_val: 84.14,      Loss_max_val: 171.67 
      Loss_mean_trainval: 118.05
Epoch 93/4000, Loss_mean: 105.53,      Loss_min: 84.15, Loss_max: 125.63 
      Loss_mean_val: 118.72, Loss_min_val: 78.07,      Loss_max_val: 158.4 
      Loss_mean_trainval: 117.31
Epoch 94/4000, Loss_mean: 101.85,      Loss_min: 62.49, Loss_max: 141.66 
      Loss_mean_val: 121.56, Loss_min_val: 62.33,      Loss_max_val: 193.06 
      Loss_mean_trainval: 119.45
Epoch 95/4000, Loss_mean: 96.34,      Loss_min: 68.59, Loss_max: 127.95 
      Loss_mean_val: 118.78, Loss_min_val: 74.89,      Loss_max_val: 184.18 
      Loss_mean_trainval: 116.37
Epoch 96/4000, Loss_mean: 103.58,      Loss_min: 83.48, Loss_max: 121.31 
      Loss

Epoch 136/4000, Loss_mean: 83.57,      Loss_min: 73.54, Loss_max: 94.99 
      Loss_mean_val: 105.85, Loss_min_val: 58.84,      Loss_max_val: 146.67 
      Loss_mean_trainval: 103.46
Epoch 137/4000, Loss_mean: 80.85,      Loss_min: 67.3, Loss_max: 98.61 
      Loss_mean_val: 106.27, Loss_min_val: 66.69,      Loss_max_val: 166.91 
      Loss_mean_trainval: 103.55
Epoch 138/4000, Loss_mean: 90.7,      Loss_min: 59.32, Loss_max: 149.66 
      Loss_mean_val: 106.69, Loss_min_val: 65.03,      Loss_max_val: 142.56 
      Loss_mean_trainval: 104.98
Epoch 139/4000, Loss_mean: 78.83,      Loss_min: 63.8, Loss_max: 94.72 
      Loss_mean_val: 105.22, Loss_min_val: 70.42,      Loss_max_val: 152.87 
      Loss_mean_trainval: 102.4
Epoch 140/4000, Loss_mean: 82.68,      Loss_min: 65.05, Loss_max: 99.61 
      Loss_mean_val: 105.14, Loss_min_val: 64.89,      Loss_max_val: 143.96 
      Loss_mean_trainval: 102.73
Epoch 141/4000, Loss_mean: 78.83,      Loss_min: 42.68, Loss_max: 104.37 
      Loss_mea

Epoch 182/4000, Loss_mean: 70.28,      Loss_min: 53.31, Loss_max: 92.92 
      Loss_mean_val: 96.58, Loss_min_val: 59.31,      Loss_max_val: 154.93 
      Loss_mean_trainval: 93.76
Epoch 183/4000, Loss_mean: 68.8,      Loss_min: 55.59, Loss_max: 83.64 
      Loss_mean_val: 93.88, Loss_min_val: 56.95,      Loss_max_val: 154.23 
      Loss_mean_trainval: 91.19
Epoch 184/4000, Loss_mean: 71.41,      Loss_min: 47.87, Loss_max: 90.35 
      Loss_mean_val: 96.19, Loss_min_val: 61.49,      Loss_max_val: 132.36 
      Loss_mean_trainval: 93.53
Epoch 185/4000, Loss_mean: 67.08,      Loss_min: 51.68, Loss_max: 93.02 
      Loss_mean_val: 92.35, Loss_min_val: 42.97,      Loss_max_val: 147.55 
      Loss_mean_trainval: 89.65
Epoch 186/4000, Loss_mean: 68.0,      Loss_min: 58.23, Loss_max: 76.04 
      Loss_mean_val: 96.9, Loss_min_val: 55.14,      Loss_max_val: 161.5 
      Loss_mean_trainval: 93.8
Epoch 187/4000, Loss_mean: 67.45,      Loss_min: 54.11, Loss_max: 80.68 
      Loss_mean_val: 93.67,

...

Epoch 3770/4000, Loss_mean: 3.32,      Loss_min: 2.11, Loss_max: 4.34 
      Loss_mean_val: 85.7, Loss_min_val: 19.85,      Loss_max_val: 155.47 
      Loss_mean_trainval: 76.88
Epoch 3771/4000, Loss_mean: 3.17,      Loss_min: 2.49, Loss_max: 4.0 
      Loss_mean_val: 82.02, Loss_min_val: 30.51,      Loss_max_val: 198.44 
      Loss_mean_trainval: 73.58
Epoch 3772/4000, Loss_mean: 3.08,      Loss_min: 2.54, Loss_max: 3.99 
      Loss_mean_val: 85.38, Loss_min_val: 30.83,      Loss_max_val: 212.12 
      Loss_mean_trainval: 76.57
Epoch 3773/4000, Loss_mean: 2.96,      Loss_min: 2.58, Loss_max: 3.71 
      Loss_mean_val: 83.39, Loss_min_val: 38.52,      Loss_max_val: 206.24 
      Loss_mean_trainval: 74.78
Epoch 3774/4000, Loss_mean: 3.36,      Loss_min: 1.84, Loss_max: 5.47 
      Loss_mean_val: 84.91, Loss_min_val: 20.79,      Loss_max_val: 147.64 
      Loss_mean_trainval: 76.18
Epoch 3775/4000, Loss_mean: 3.08,      Loss_min: 2.48, Loss_max: 3.91 
      Loss_mean_val: 82.99, Loss_min

Epoch 3816/4000, Loss_mean: 3.01,      Loss_min: 2.28, Loss_max: 3.63 
      Loss_mean_val: 84.73, Loss_min_val: 35.33,      Loss_max_val: 175.12 
      Loss_mean_trainval: 75.98
Epoch 3817/4000, Loss_mean: 3.26,      Loss_min: 2.2, Loss_max: 4.29 
      Loss_mean_val: 85.32, Loss_min_val: 23.46,      Loss_max_val: 190.89 
      Loss_mean_trainval: 76.53
Epoch 3818/4000, Loss_mean: 3.13,      Loss_min: 2.12, Loss_max: 4.19 
      Loss_mean_val: 84.1, Loss_min_val: 30.86,      Loss_max_val: 163.91 
      Loss_mean_trainval: 75.43
Epoch 3819/4000, Loss_mean: 3.27,      Loss_min: 2.04, Loss_max: 4.15 
      Loss_mean_val: 83.87, Loss_min_val: 33.44,      Loss_max_val: 156.47 
      Loss_mean_trainval: 75.24
Epoch 3820/4000, Loss_mean: 3.37,      Loss_min: 2.18, Loss_max: 4.96 
      Loss_mean_val: 85.65, Loss_min_val: 20.61,      Loss_max_val: 174.51 
      Loss_mean_trainval: 76.84
Epoch 3821/4000, Loss_mean: 3.12,      Loss_min: 2.66, Loss_max: 3.67 
      Loss_mean_val: 84.96, Loss_min

Epoch 3862/4000, Loss_mean: 3.02,      Loss_min: 2.4, Loss_max: 4.08 
      Loss_mean_val: 84.22, Loss_min_val: 35.11,      Loss_max_val: 188.21 
      Loss_mean_trainval: 75.52
Epoch 3863/4000, Loss_mean: 3.12,      Loss_min: 2.07, Loss_max: 3.84 
      Loss_mean_val: 83.56, Loss_min_val: 36.2,      Loss_max_val: 174.28 
      Loss_mean_trainval: 74.95
Epoch 3864/4000, Loss_mean: 3.11,      Loss_min: 2.48, Loss_max: 4.26 
      Loss_mean_val: 81.67, Loss_min_val: 32.06,      Loss_max_val: 190.72 
      Loss_mean_trainval: 73.26
Epoch 3865/4000, Loss_mean: 3.09,      Loss_min: 1.97, Loss_max: 4.73 
      Loss_mean_val: 83.8, Loss_min_val: 40.14,      Loss_max_val: 206.48 
      Loss_mean_trainval: 75.16
Epoch 3866/4000, Loss_mean: 2.95,      Loss_min: 1.97, Loss_max: 3.68 
      Loss_mean_val: 83.69, Loss_min_val: 35.33,      Loss_max_val: 191.31 
      Loss_mean_trainval: 75.04
Epoch 3867/4000, Loss_mean: 2.93,      Loss_min: 1.77, Loss_max: 4.57 
      Loss_mean_val: 85.98, Loss_min_

Epoch 3909/4000, Loss_mean: 3.0,      Loss_min: 2.45, Loss_max: 4.14 
      Loss_mean_val: 84.53, Loss_min_val: 31.46,      Loss_max_val: 180.92 
      Loss_mean_trainval: 75.8
Epoch 3910/4000, Loss_mean: 3.35,      Loss_min: 1.9, Loss_max: 4.21 
      Loss_mean_val: 83.61, Loss_min_val: 20.77,      Loss_max_val: 178.76 
      Loss_mean_trainval: 75.01
Epoch 3911/4000, Loss_mean: 3.02,      Loss_min: 2.35, Loss_max: 3.54 
      Loss_mean_val: 83.69, Loss_min_val: 20.4,      Loss_max_val: 181.21 
      Loss_mean_trainval: 75.05
Epoch 3912/4000, Loss_mean: 2.85,      Loss_min: 1.23, Loss_max: 4.42 
      Loss_mean_val: 86.69, Loss_min_val: 24.44,      Loss_max_val: 142.41 
      Loss_mean_trainval: 77.72
Epoch 3913/4000, Loss_mean: 2.94,      Loss_min: 2.68, Loss_max: 3.44 
      Loss_mean_val: 84.14, Loss_min_val: 25.08,      Loss_max_val: 236.66 
      Loss_mean_trainval: 75.45
Epoch 3914/4000, Loss_mean: 2.99,      Loss_min: 2.34, Loss_max: 3.96 
      Loss_mean_val: 84.95, Loss_min_v

Epoch 3955/4000, Loss_mean: 3.34,      Loss_min: 2.09, Loss_max: 4.3 
      Loss_mean_val: 86.95, Loss_min_val: 29.5,      Loss_max_val: 234.03 
      Loss_mean_trainval: 78.0
Epoch 3956/4000, Loss_mean: 3.04,      Loss_min: 2.43, Loss_max: 3.91 
      Loss_mean_val: 83.43, Loss_min_val: 23.84,      Loss_max_val: 157.13 
      Loss_mean_trainval: 74.82
Epoch 3957/4000, Loss_mean: 2.93,      Loss_min: 2.25, Loss_max: 3.6 
      Loss_mean_val: 86.2, Loss_min_val: 20.93,      Loss_max_val: 165.95 
      Loss_mean_trainval: 77.29
Epoch 3958/4000, Loss_mean: 2.77,      Loss_min: 2.1, Loss_max: 3.04 
      Loss_mean_val: 86.24, Loss_min_val: 42.09,      Loss_max_val: 167.26 
      Loss_mean_trainval: 77.31
Epoch 3959/4000, Loss_mean: 2.89,      Loss_min: 2.63, Loss_max: 3.34 
      Loss_mean_val: 87.82, Loss_min_val: 34.26,      Loss_max_val: 204.38 
      Loss_mean_trainval: 78.73
Epoch 3960/4000, Loss_mean: 2.79,      Loss_min: 2.35, Loss_max: 3.61 
      Loss_mean_val: 84.88, Loss_min_val